In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F

In [74]:
class Trainset(Dataset):
    def __init__(self, path,device):
        self.imgs = []
        self.device=device
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Resize((512,512)),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        for root, _, files in os.walk(path):
            for file in files:
                self.imgs.append(os.path.join(root, file))

    def __getitem__(self, index):
        x=self.imgs[index]
        y=0
        if 'dog.' in x:
            y=1
        image=Image.open(x).convert('RGB')
        image=self.transform(image).unsqueeze(0)
        image=image.resize(3,512,512)
        return image.to(self.device,torch.float),y

    def __len__(self):
        return len(self.imgs)

In [155]:
class Testset(Dataset):
    def __init__(self, path,device):
        self.imgs = []
        self.device=device
        self.transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Resize((512,512)),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        for root, _, files in os.walk(path):
            for file in files:
                self.imgs.append(os.path.join(root, file))

    def __getitem__(self, index):
        x=self.imgs[index]
        image=Image.open(x).convert('RGB')
        image=self.transform(image).unsqueeze(0)
        id=x.split('/')[-1].split('.')[0]
        image=image.resize(3,512,512)
        return image.to(self.device,torch.float),id

    def __len__(self):
        return len(self.imgs)

In [128]:
trainset = Trainset('./train/', 'cpu')
trainloader = DataLoader(trainset, batch_size=4,
                         shuffle=True)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 32)
        self.fc1 = nn.Linear(692224//4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 692224//4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [129]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [132]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

c:\python37\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


[1,   200] loss: 0.070
[1,   400] loss: 0.069
[1,   600] loss: 0.069
[1,   800] loss: 0.069
[1,  1000] loss: 0.069
[1,  1200] loss: 0.069
[1,  1400] loss: 0.068
[1,  1600] loss: 0.067
[1,  1800] loss: 0.067
[1,  2000] loss: 0.069
[1,  2200] loss: 0.067
[1,  2400] loss: 0.066
[1,  2600] loss: 0.068
[1,  2800] loss: 0.067
[1,  3000] loss: 0.067
[1,  3200] loss: 0.068
[1,  3400] loss: 0.067
[1,  3600] loss: 0.068
[1,  3800] loss: 0.066
[1,  4000] loss: 0.068
[1,  4200] loss: 0.065
[1,  4400] loss: 0.068
[1,  4600] loss: 0.065
[1,  4800] loss: 0.065
[1,  5000] loss: 0.064
[1,  5200] loss: 0.065
[1,  5400] loss: 0.067
[1,  5600] loss: 0.064
[1,  5800] loss: 0.065
[1,  6000] loss: 0.065
[1,  6200] loss: 0.065
[2,   200] loss: 0.062
[2,   400] loss: 0.063
[2,   600] loss: 0.065
[2,   800] loss: 0.065
[2,  1000] loss: 0.065
[2,  1200] loss: 0.063
[2,  1400] loss: 0.064
[2,  1600] loss: 0.063
[2,  1800] loss: 0.067
[2,  2000] loss: 0.064
[2,  2200] loss: 0.065
[2,  2400] loss: 0.064
[2,  2600] 

In [133]:
torch.save(net, './model') 

In [156]:
testset=Testset('./test/', 'cpu')
testloader = DataLoader(testset, batch_size=4,shuffle=False)
all_outputs=[]
all_ids=[]
with torch.no_grad():
    for data in testloader:
        images, ids = data
        outputs = net(images)
        all_outputs.extend(outputs)
        all_ids.extend(ids)

c:\python37\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [157]:
import numpy as np
all_outputs=[x.numpy() for x in all_outputs]
all_outputs=np.array(all_outputs)

In [158]:
l=np.argmax(all_outputs,axis=1)

In [160]:
f=open('result.csv','w',encoding='utf8')
f.write('id,label\n')
for id,label in zip(all_ids,l):
    f.write(id+','+str(label)+'\n')
f.close()